In [ ]:
grid = build_grid([nPixels,nPixels])

In [ ]:
grid.shape

In [ ]:
fig,axs =plt.subplots(1,4,figsize=(10,2))
for i,ax in enumerate(axs):
    ax.imshow(grid[0,:,:,i],origin='lower',extent=[xlow,xhigh]*2)
    
plt.show()

In [ ]:
Y = regCNN.cnn(X)

In [ ]:
grid = build_grid([nPixels,nPixels])    

In [ ]:
class SoftPositionEmbed(torch.nn.Module):
    '''
    Translating the google fct from tf into pytorch:
    https://github.com/nhartman94/google-research/blob/master/slot_attention/model.py#L367-L382
    '''
    def __init__(self, hidden_dim, resolution,device='cpu'):
        super().__init__()
        
        self.dense = torch.nn.Linear(4,hidden_dim)
        self.grid = torch.FloatTensor(build_grid(resolution)).to(device)
    
    def forward(self, X):
    
        return X + torch.permute(self.dense(self.grid),(0,3,1,2) )
    

In [ ]:
class posEmbedCNN(torch.nn.Module):
    '''
    CNN to regress the circle properties:
    - position (x,y)
    - radiu
    '''
    def __init__(self, hidden_dim=hidden_dim, mlp_dim=64, device='cpu'):
        '''
        '''
        super().__init__()
        
        self.cnn = torch.nn.Sequential(
                    torch.nn.Conv2d(1, hidden_dim, 5, padding=2),
                    torch.nn.ReLU(),
                    torch.nn.Conv2d(hidden_dim, hidden_dim, 5, padding=2),
#                     torch.nn.ReLU(),
#                     torch.nn.Conv2d(hidden_dim, 1, 5, padding=2),

            )
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(nPixels*nPixels,mlp_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(mlp_dim,3)
        )
        
        
        self.enc_pos = SoftPositionEmbed(hidden_dim,[nPixels]*2,device=device)
        
    def forward(self,X):
        Y = self.cnn(X)
        
        Y = self.enc_pos(Y)
        
        Y = torch.flatten(Y,1)
        Y = self.mlp(Y)
        
        # As per Florian's suggestion... predict \log sigma^2
        Y[:,-1] = torch.exp(Y[:,-1])
        
        return Y

In [ ]:
posEmb = posEmbedCNN(device=device).to(device)

In [ ]:
out = posEmb(X)